# Lab-11-3 RNN Long sequence

In [1]:
import numpy as np
import torch
from torch import nn
from torch import optim

In [2]:
sentence = open("./texts/johndenver_countryroads_lyrics.txt", 'r')

In [3]:
text = sentence.readline()
print(text)

Almost heaven, West Virginia Blue Ridge Mountains, Shenandoah River Life is old there, older than the trees Younger than the mountains, growin' like a breeze Country roads, take me home To the place I belong West Virginia, mountain mama Take me home, country roads All my memories gather 'round her Miner's lady, stranger to blue water Dark and dusty, painted on the sky Misty taste of moonshine, teardrop in my eye Country roads, take me home To the place I belong West Virginia, mountain mama Take me home, country roads I hear her voice in the mornin' hour, she calls me The radio reminds me of my home far away Driving down the road, I get a feeling That I should've been home yesterday, yesterday Country roads, take me home To the place I belong West Virginia, mountain mama Take me home, country roads Country roads, take me home To the place I belong West Virginia, mountain mama Take me home, country roads


In [4]:
text = text.replace(',', '')\
           .replace('.', '')\
           .replace('"', '')\
           .replace("'", '')\
           .lower()
text

'almost heaven west virginia blue ridge mountains shenandoah river life is old there older than the trees younger than the mountains growin like a breeze country roads take me home to the place i belong west virginia mountain mama take me home country roads all my memories gather round her miners lady stranger to blue water dark and dusty painted on the sky misty taste of moonshine teardrop in my eye country roads take me home to the place i belong west virginia mountain mama take me home country roads i hear her voice in the mornin hour she calls me the radio reminds me of my home far away driving down the road i get a feeling that i shouldve been home yesterday yesterday country roads take me home to the place i belong west virginia mountain mama take me home country roads country roads take me home to the place i belong west virginia mountain mama take me home country roads'

In [5]:
char_set = list(set(text))
char_dic = {c: i for i, c in enumerate(char_set)}

In [6]:
dic_size = len(char_dic)

In [7]:
x_data = []
y_data = []

In [8]:
seq_length = 40

In [9]:
for i in range(len(text) - seq_length):
    x_str = text[i : i+seq_length]
    y_str = text[i+1 : i+seq_length+1]
    print(i, x_str, '>>>', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
    break

0 almost heaven west virginia blue ridge m >>> lmost heaven west virginia blue ridge mo


In [10]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

In [11]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

## Hyperparameters

In [12]:
input_size = dic_size
hidden_size = dic_size
learning_rate = 1e-1

## Model

In [13]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [14]:
net = Net(input_size, hidden_size, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

## Train

In [15]:
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    predict_str = ""
    if epoch % 10 == 0:
        for j, result in enumerate(results):
            print('Epoch:', epoch, 'Loss:', loss.item())
            print(j, ''.join([char_set[t] for t in result]))
            if j == 0:
                predict_str += ''.join([char_set[t] for t in result])
            else:
                predict_str += char_set[result[-1]]

Epoch: 0 Loss: 3.178008556365967
0 v ffff  fffff  fff  ff ffff f fs     f  
Epoch: 10 Loss: 1.8780879974365234
0 lmost me ion t it mon e me mn e sense mo
Epoch: 20 Loss: 0.49211448431015015
0 lmost heameniwest ridgenma blue ridgenmo
Epoch: 30 Loss: 0.08612872660160065
0 lmost heaven west virginia blue ridge mo
Epoch: 40 Loss: 0.021281754598021507
0 lmost heaven west virginia blue ridge mo
Epoch: 50 Loss: 0.007485988549888134
0 lmost heaven west virginia blue ridge mo
Epoch: 60 Loss: 0.003886774880811572
0 lmost heaven west virginia blue ridge mo
Epoch: 70 Loss: 0.002695688745006919
0 lmost heaven west virginia blue ridge mo
Epoch: 80 Loss: 0.0021736605558544397
0 lmost heaven west virginia blue ridge mo
Epoch: 90 Loss: 0.0018448485061526299
0 lmost heaven west virginia blue ridge mo


## close()

In [16]:
sentence.close()